In [1]:
import csv
import re
import requests
from collections import defaultdict
from xml.etree import ElementTree as ET
from time import sleep

In [2]:
REGEX_TAGS_TO_REMOVE = r'<tbody class="\w+">|<.?tbody>|<.?thead>|&nbsp;|<caption .*</caption>|<a href=".+">|<\\a>'
REGEX_VOTING_TABLE = r'<table class="kluby">.+</table>'
REGEX_CLUBS = r'<a href="agent.xsp\?symbol=klubglos&IdGlosowania=\d+&KodKlubu=\w+\-?\w+\.?"><strong>(\w+\-?\w+\.?)</strong></a>'
REGEX_VOTES = r'<a href="agent.xsp\?symbol=klubdecglos&IdGlosowania=\d+&KodKlubu=\w+\-?\w+\.?&Decyzja=\w+"><strong>(\d+)</strong></a>'
REGEX_VOTE_URL = r'<a class="vote" href="(\w+\.\w+\?symbol=glosowania&nrkadencji=\d+&nrposiedzenia=\d+&nrglosowania=\d+)">\(głos\.\s\w+\s\d+\)</a></p><p>Decyzja: uchwalono</p>'
REGEX_STATUTE = r'PrzebiegProc\.xsp\?nr=\d+'

In [5]:
HEADER_PARTY = 'Klub'
HEADER_TOTAL = 'Liczba'
HEADER_VOTED = 'Glosowalo'
HEADER_FOR = 'Za'
HEADER_AGAINST = 'Przeciw'
HEADER_ABSTAINED = 'Wstrzym'
TRANSLATION_TABLE = {'</td></td>': '</td></tr>', 'Klub/Koło': HEADER_PARTY, 'Liczba czł.': HEADER_TOTAL, 'Głosowało': HEADER_VOTED, ' Za ': HEADER_FOR, 'Przeciw': HEADER_AGAINST, 'Wstrzymało się': HEADER_ABSTAINED, 'Nie głosowało': HEADER_ABSTAINED}

URL_PATTERN = 'https://www.sejm.gov.pl/Sejm9.nsf/{}'

In [6]:
THRESHOLD = 0.65

In [7]:
def clean_table(s):
    for phrase in TRANSLATION_TABLE:
        s = re.sub(phrase, TRANSLATION_TABLE[phrase], s)

    for group in re.findall(REGEX_TAGS_TO_REMOVE, s):
        s = re.sub(group, '', s)
    
    s = re.sub(REGEX_CLUBS, r'\1', s)
    s = re.sub(REGEX_VOTES, r'\1', s)
    
    return s

In [8]:
def get_voting_results(cleaned_table):
    ### Adapted from: https://stackoverflow.com/a/7315891 CC-BY-SA 4.0
    res = []
    table = ET.XML(cleaned_table)
    rows = iter(table)
    headers = [col.text for col in next(rows)]
    for row in rows:
        values = [int(col.text) if col.text.isdigit() else col.text for col in row]
        for i in range(len(values)):
            if values[i] == '-':
                values[i] = 0
        res.append(dict(zip(headers, values)))
    return res

In [9]:
def get_voting_table(page):
    return re.findall(REGEX_VOTING_TABLE, page)[0]

In [10]:
def get_table_for_against(results):
    table_for_against = {}

    for club in results:
        members = club[HEADER_TOTAL]
        votes_for = club[HEADER_FOR]
        votes_total = club[HEADER_VOTED]
        res = votes_for >= votes_total * THRESHOLD and votes_total >= members * THRESHOLD
        table_for_against[club[HEADER_PARTY]] = res

    return table_for_against

In [11]:
def voted_with(table, who):
    co_voters = []
    expected = table[who]
    for party in table:
        if party != who and table[party] == expected:
            co_voters.append(party)
    return co_voters

In [12]:
def get_voting_page(statute_url):
    statute_page = requests.get(statute_url).text
    sleep(0.5)
    url_end = re.findall(REGEX_VOTE_URL, statute_page)[0]
    url = URL_PATTERN.format(url_end)
    voting_page = requests.get(url).text
    return voting_page

In [13]:
def get_voting_page_results(voting_page):
    page = get_voting_table(voting_page)
    results = get_voting_results(clean_table(page))
    return results

In [15]:
url_all_statutes = 'https://www.sejm.gov.pl/Sejm9.nsf/agent.xsp?symbol=USTAWYALL&NrKadencji=9&NrPosiedzenia=28'
all_statutes = requests.get(url_all_statutes).text
urls = re.findall(REGEX_STATUTE, all_statutes)

In [17]:
raw_results = {}

In [19]:
i = 1

for url in urls:
    try:
        statute = URL_PATTERN.format(url)
        print("Getting {}: {}...".format(i, statute))
        i += 1
        voting_page = get_voting_page(statute)
        print("Sleeping...")
        sleep(1.2)
        results = get_voting_page_results(voting_page)
        raw_results[url] = results
    except:
        print("ERROR {}".format(url))
    sleep(0.5)

Getting 1: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=867...
Sleeping...
Getting 2: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=899...
Sleeping...
Getting 3: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=916...
Sleeping...
Getting 4: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=957...
Sleeping...
Getting 5: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=985...
Sleeping...
Getting 6: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=990...
Sleeping...
Getting 7: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=1016...
Sleeping...
Getting 8: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=808...
Sleeping...
Getting 9: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=969...
Sleeping...
Getting 10: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=970...
Sleeping...
Getting 11: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=1008...
Sleeping...
Getting 12: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=1012..

Sleeping...
Getting 98: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=457...
Sleeping...
Getting 99: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=590...
Sleeping...
Getting 100: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=378...
Sleeping...
Getting 101: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=133...
Sleeping...
Getting 102: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=260...
Sleeping...
Getting 103: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=420...
Sleeping...
Getting 104: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=559...
Sleeping...
Getting 105: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=562...
Sleeping...
Getting 106: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=573...
Sleeping...
Getting 107: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=605...
Sleeping...
Getting 108: https://www.sejm.gov.pl/Sejm9.nsf/PrzebiegProc.xsp?nr=388...
Sleeping...
Getting 109: https://www.sejm.gov.pl/Sejm9.n

In [30]:
all_tables_for_against = []

for statute in list(raw_results.values()):
    tabled = get_table_for_against(statute)
    all_tables_for_against.append(tabled)
    

In [42]:
co_voters = []
PiS_co_voters = defaultdict(int)

for table in all_tables_for_against:
    co_voters = []
    expected = table['PiS']
    for party in table:
        if party != 'PiS' and table[party] == expected:
            co_voters.append(party)
    for party in co_voters:
        PiS_co_voters[party] += 1


In [44]:
co_voters = []
KO_co_voters = defaultdict(int)

for table in all_tables_for_against:
    co_voters = []
    expected = table['KO']
    for party in table:
        if party != 'KO' and table[party] == expected:
            co_voters.append(party)
    for party in co_voters:
        KO_co_voters[party] += 1

In [48]:
vote_participation = defaultdict(int)
for table in all_tables_for_against:
    for party in table:
        vote_participation[party] += 1

In [49]:
print(vote_participation)

defaultdict(<class 'int'>, {'PiS': 190, 'KO': 190, 'Lewica': 190, 'KP': 60, 'Konfederacja': 190, 'Kukiz15': 37, 'niez.': 190, 'Polska2050': 37, 'PSL-Kukiz15': 130})


In [43]:
print(PiS_co_voters)

defaultdict(<class 'int'>, {'KO': 129, 'Lewica': 138, 'KP': 46, 'Kukiz15': 22, 'Polska2050': 25, 'niez.': 130, 'Konfederacja': 61, 'PSL-Kukiz15': 90})


In [45]:
print(KO_co_voters)

defaultdict(<class 'int'>, {'Lewica': 171, 'KP': 50, 'Konfederacja': 98, 'Kukiz15': 26, 'niez.': 165, 'Polska2050': 35, 'PiS': 129, 'PSL-Kukiz15': 107})


In [62]:
pattern = "Posłowie i posłanki z {} (jako klub/koło/zbiorowość) głosowali tak samo, jak {} w {:.2f}% swoich głosowań."


for other_party in ('PiS', 'KO'):
    for party in vote_participation:
        if party != other_party:
            num = PiS_co_voters if other_party == 'PiS' else KO_co_voters
            den = vote_participation
            percent = num[party] / den[party] * 100
            print(pattern.format(party, other_party, percent))
    print('-' * 100)
    

Posłowie i posłanki z KO (jako klub/koło/zbiorowość) głosowali tak samo, jak PiS w 67.89% swoich głosowań.
Posłowie i posłanki z Lewica (jako klub/koło/zbiorowość) głosowali tak samo, jak PiS w 72.63% swoich głosowań.
Posłowie i posłanki z KP (jako klub/koło/zbiorowość) głosowali tak samo, jak PiS w 76.67% swoich głosowań.
Posłowie i posłanki z Konfederacja (jako klub/koło/zbiorowość) głosowali tak samo, jak PiS w 32.11% swoich głosowań.
Posłowie i posłanki z Kukiz15 (jako klub/koło/zbiorowość) głosowali tak samo, jak PiS w 59.46% swoich głosowań.
Posłowie i posłanki z niez. (jako klub/koło/zbiorowość) głosowali tak samo, jak PiS w 68.42% swoich głosowań.
Posłowie i posłanki z Polska2050 (jako klub/koło/zbiorowość) głosowali tak samo, jak PiS w 67.57% swoich głosowań.
Posłowie i posłanki z PSL-Kukiz15 (jako klub/koło/zbiorowość) głosowali tak samo, jak PiS w 69.23% swoich głosowań.
----------------------------------------------------------------------------------------------------
Posł